In [177]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import re
import numpy as np

In [178]:
path = '../data/_data.csv'
df = pd.read_csv(path)
pd.set_option('display.max_columns', None)

In [179]:
df.iloc[:, :13]

,Unnamed: 0,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт
0,0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский
1,1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский
2,2,271173086,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт
3,3,272197456,"4, Оба варианта",Квартира,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт
4,4,273614615,2,Квартира,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23363,23363,215565511,NaN,Квартира,м. Говорово (8 мин пешком),"Москва, Боровское шоссе, 2к7, ш. Боровское (3 ...",35.0/16.4/8.0,10/14,NaN,"42000.0 руб./ За месяц, Залог - 42000 руб., Ко...",+79154383401,Сдаётся светлая теплая квартира (студия) с бал...,Евроремонт
23364,23364,274654844,1,Квартира,м. Солнцево (7 мин пешком),"Москва, Производственная улица, 8к1, ш. Боровс...",38.7/16.5/11.0,"5/18, Монолитный",NaN,"45000.0 руб./ За месяц, Залог - 45000 руб., Ко...",+79151173402,Сдам однокомнатную квартиру на длительный срок...,Евроремонт
23365,23365,268679909,"2, Оба варианта",Квартира,м. Солнцево (6 мин пешком),"Москва, Боровский проезд, 11",43.1,"5/5, Кирпичный",NaN,"50000.0 руб./ За месяц, Залог - 50000 руб., Ко...",+79653825192,Рассматриваются квартиранты только славяне и т...,Дизайнерский
23366,23366,274807525,2,Квартира,м. Солнцево (11 мин пешком),"Москва, улица Богданова, 6к1, ш. Боровское (3 ...",52.5/10.0,"8/23, Монолитный",наземная,"55000.0 руб./ За месяц, Залог - 50000 руб., Ко...",+79660611965,Двухкомнатная уютная квартира с видом во внутр...,Евроремонт


In [180]:
# удаляем столбец 'Unnamed: 0'
df = df.drop('Unnamed: 0',axis = 1)
# убираем адреса не в Мск
df = df[df['Адрес'].str.contains('Москва')]
# удаляем пропуски в колонке Метро
df = df.dropna(subset=['Метро'])
df.iloc[:, :13]

,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт,"Площадь комнат, м2"
0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN
1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25
2,271173086,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN
3,272197456,"4, Оба варианта",Квартира,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19
4,273614615,2,Квартира,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23363,215565511,NaN,Квартира,м. Говорово (8 мин пешком),"Москва, Боровское шоссе, 2к7, ш. Боровское (3 ...",35.0/16.4/8.0,10/14,NaN,"42000.0 руб./ За месяц, Залог - 42000 руб., Ко...",+79154383401,Сдаётся светлая теплая квартира (студия) с бал...,Евроремонт,NaN
23364,274654844,1,Квартира,м. Солнцево (7 мин пешком),"Москва, Производственная улица, 8к1, ш. Боровс...",38.7/16.5/11.0,"5/18, Монолитный",NaN,"45000.0 руб./ За месяц, Залог - 45000 руб., Ко...",+79151173402,Сдам однокомнатную квартиру на длительный срок...,Евроремонт,NaN
23365,268679909,"2, Оба варианта",Квартира,м. Солнцево (6 мин пешком),"Москва, Боровский проезд, 11",43.1,"5/5, Кирпичный",NaN,"50000.0 руб./ За месяц, Залог - 50000 руб., Ко...",+79653825192,Рассматриваются квартиранты только славяне и т...,Дизайнерский,NaN
23366,274807525,2,Квартира,м. Солнцево (11 мин пешком),"Москва, улица Богданова, 6к1, ш. Боровское (3 ...",52.5/10.0,"8/23, Монолитный",наземная,"55000.0 руб./ За месяц, Залог - 50000 руб., Ко...",+79660611965,Двухкомнатная уютная квартира с видом во внутр...,Евроремонт,NaN


In [181]:
# преобразование цены 
def get_price(y):
  r = ''
  for i in y:
    if i.isdigit():
      r += i
    else:
      r += ' '
  return int(r.split()[0])

def get_deposit(y):
  r = ''
  for i in y:
    if i.isdigit():
      r += i
    else:
      r += ' '
  try:
      if int(r.split()[2]) < 1000 or int(r.split()[2]) > int(r.split()[0]) * 6:
        return int(r.split()[0])
      else:
        return int(r.split()[2])
  except:
      return int(r.split()[0])
    
x1 = df['Цена'].map(get_price) 
x2 = df['Цена'].map(get_deposit) 

In [182]:
df.insert(9, "Цена за месяц",x1)
df.insert(10,'Залог',x2)
df.head(3)

,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Цена за месяц,Залог,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",500000,500000,+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",500000,500000,+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126
2,271173086,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",500000,500000,"+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086


In [183]:
# преобразование этажа и этажности дома

def first_element(x):
    y = re.findall(r'\d+', x)
    return y[0]

df['Этаж'] = df['Дом'].map(first_element).astype('int')
def second_element(x):
    y = re.findall(r'\d+', x)
    return y[1]

df['Этажность дома'] = df['Дом'].apply(second_element).astype('int')
from operator import methodcaller
def func(x):
  return float(x[0])
point_place = df['Площадь, м2'].map(methodcaller("split", "/")).map(func).copy()
df.insert(5, "Общая площадь",point_place)

In [184]:
def func (y):
    y[:y.find('(')]
    return y[:y.find('(')].strip()

def func2 (y):
    y[:y.find('(')]
    return y[y.find('м. ')+3:].strip()

x = df['Метро'].map(func).map(func2).copy()
df.insert(3, "Станция", x)

In [185]:
df.head(4)

,ID объявления,Количество комнат,Тип,Станция,Метро,Адрес,Общая площадь,"Площадь, м2",Дом,Парковка,Цена,Цена за месяц,Залог,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление,Этаж,Этажность дома
0,271271157,4,Квартира,Смоленская,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0,200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",500000,500000,+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157,5,16
1,271634126,4,Квартира,Смоленская,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0,198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",500000,500000,+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126,5,16
2,271173086,"4, Оба варианта",Квартира,Смоленская,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0,200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",500000,500000,"+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086,5,16
3,272197456,"4, Оба варианта",Квартира,Смоленская,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0,170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",400000,400000,"+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456,5,6


In [186]:

df.drop(['Метро', 'Цена','Описание','Дополнительно','Название ЖК'], axis= 1, inplace=True)
df = df[['ID  объявления','Адрес','Станция','Серия дома', 'Общая площадь',	'Площадь, м2', 'Этаж',	'Этажность дома', 'Высота потолков, м', 'Площадь комнат, м2', 'Количество комнат','Ремонт','Балкон', 'Окна','Санузел','Лифт','Мусоропровод', 'Тип','Дом',	'Парковка',	'Цена за месяц','Залог','Можно с детьми/животными']].copy()
df = df.rename(columns={'ID  объявления':'ID'}).reset_index(drop=True)

In [187]:
df 

,ID,Адрес,Станция,Серия дома,Общая площадь,"Площадь, м2",Этаж,Этажность дома,"Высота потолков, м","Площадь комнат, м2",Количество комнат,Ремонт,Балкон,Окна,Санузел,Лифт,Мусоропровод,Тип,Дом,Парковка,Цена за месяц,Залог,Можно с детьми/животными
0,271271157,"Москва, улица Новый Арбат, 27",Смоленская,NaN,200.0,200.0/20.0,5,16,3.00,NaN,4,Дизайнерский,NaN,NaN,NaN,"Пасс (4), Груз (1)",Да,Квартира,"5/16, Монолитный",подземная,500000,500000,"Можно с детьми, Можно с животными"
1,271634126,"Москва, улица Новый Арбат, 27",Смоленская,NaN,198.0,198.0/95.0/18.0,5,16,3.50,25 25 20 25,4,Дизайнерский,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)","Пасс (1), Груз (1)",Нет,Квартира,"5/16, Монолитно-кирпичный",подземная,500000,500000,Можно с детьми
2,271173086,"Москва, улица Новый Арбат, 27",Смоленская,NaN,200.0,200.0/116.0/4.0,5,16,3.20,NaN,"4, Оба варианта",Евроремонт,NaN,На улицу и двор,Совмещенный (3),Пасс (1),NaN,Квартира,5/16,подземная,500000,500000,Можно с детьми
3,272197456,"Москва, переулок Плотников, 21С1",Смоленская,NaN,170.0,170.0/95.0/17.0,5,6,3.20,14-42-20-19,"4, Оба варианта",Евроремонт,NaN,На улицу и двор,Совмещенный (3),Пасс (1),NaN,Квартира,5/6,подземная,400000,400000,Можно с животными
4,273614615,"Москва, улица Новый Арбат, 15",Арбатская,NaN,58.0,58.0/38.0/5.0,12,26,3.90,20 18,2,Евроремонт,NaN,На улицу и двор,Совмещенный (2),"Пасс (1), Груз (1)",Да,Квартира,"12/26, Панельный",NaN,225000,225000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19386,215565511,"Москва, Боровское шоссе, 2к7, ш. Боровское (3 ...",Говорово,NaN,35.0,35.0/16.4/8.0,10,14,NaN,NaN,NaN,Евроремонт,Балкон (1),NaN,Совмещенный (1),"Пасс (1), Груз (1)",NaN,Квартира,10/14,NaN,42000,42000,NaN
19387,274654844,"Москва, Производственная улица, 8к1, ш. Боровс...",Солнцево,NaN,38.7,38.7/16.5/11.0,5,18,NaN,NaN,1,Евроремонт,Лоджия (1),Во двор,Совмещенный (1),"Пасс (1), Груз (1)",NaN,Квартира,"5/18, Монолитный",NaN,45000,45000,NaN
19388,268679909,"Москва, Боровский проезд, 11",Солнцево,NaN,43.1,43.1,5,5,NaN,NaN,"2, Оба варианта",Дизайнерский,Балкон (1),На улицу и двор,Совмещенный (1),NaN,NaN,Квартира,"5/5, Кирпичный",NaN,50000,50000,Можно с детьми
19389,274807525,"Москва, улица Богданова, 6к1, ш. Боровское (3 ...",Солнцево,NaN,52.5,52.5/10.0,8,23,2.65,NaN,2,Евроремонт,Лоджия (1),Во двор,"Совмещенный (1), Раздельный (1)",Пасс (3),Да,Квартира,"8/23, Монолитный",наземная,55000,50000,NaN


In [188]:
df.drop(['Лифт', 'Санузел', 'Высота потолков, м', 'Балкон', 'Парковка', 'Мусоропровод', 'Окна'], axis= 1, inplace=True)

In [189]:
df

,ID,Адрес,Станция,Серия дома,Общая площадь,"Площадь, м2",Этаж,Этажность дома,"Площадь комнат, м2",Количество комнат,Ремонт,Тип,Дом,Цена за месяц,Залог,Можно с детьми/животными
0,271271157,"Москва, улица Новый Арбат, 27",Смоленская,NaN,200.0,200.0/20.0,5,16,NaN,4,Дизайнерский,Квартира,"5/16, Монолитный",500000,500000,"Можно с детьми, Можно с животными"
1,271634126,"Москва, улица Новый Арбат, 27",Смоленская,NaN,198.0,198.0/95.0/18.0,5,16,25 25 20 25,4,Дизайнерский,Квартира,"5/16, Монолитно-кирпичный",500000,500000,Можно с детьми
2,271173086,"Москва, улица Новый Арбат, 27",Смоленская,NaN,200.0,200.0/116.0/4.0,5,16,NaN,"4, Оба варианта",Евроремонт,Квартира,5/16,500000,500000,Можно с детьми
3,272197456,"Москва, переулок Плотников, 21С1",Смоленская,NaN,170.0,170.0/95.0/17.0,5,6,14-42-20-19,"4, Оба варианта",Евроремонт,Квартира,5/6,400000,400000,Можно с животными
4,273614615,"Москва, улица Новый Арбат, 15",Арбатская,NaN,58.0,58.0/38.0/5.0,12,26,20 18,2,Евроремонт,Квартира,"12/26, Панельный",225000,225000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19386,215565511,"Москва, Боровское шоссе, 2к7, ш. Боровское (3 ...",Говорово,NaN,35.0,35.0/16.4/8.0,10,14,NaN,NaN,Евроремонт,Квартира,10/14,42000,42000,NaN
19387,274654844,"Москва, Производственная улица, 8к1, ш. Боровс...",Солнцево,NaN,38.7,38.7/16.5/11.0,5,18,NaN,1,Евроремонт,Квартира,"5/18, Монолитный",45000,45000,NaN
19388,268679909,"Москва, Боровский проезд, 11",Солнцево,NaN,43.1,43.1,5,5,NaN,"2, Оба варианта",Дизайнерский,Квартира,"5/5, Кирпичный",50000,50000,Можно с детьми
19389,274807525,"Москва, улица Богданова, 6к1, ш. Боровское (3 ...",Солнцево,NaN,52.5,52.5/10.0,8,23,NaN,2,Евроремонт,Квартира,"8/23, Монолитный",55000,50000,NaN


#### Метро 


In [190]:
metro = pd.read_csv('../data/list_of_moscow_metro_stations.csv', sep=',').rename(columns={'Station':'Станция'})
metro.head()

,Станция,metro,coor
0,Третьяковская,"Москва, станция метро Третьяковская","(55.738862, 37.6284034)"
1,Медведково,"Москва, станция метро Медведково","(55.886766, 37.6615351)"
2,Первомайская,"Москва, станция метро Первомайская","(55.7931157, 37.7999286)"
3,Калужская,"Москва, станция метро Калужская","(55.6550976, 37.5421849)"
4,Каховская,"Москва, станция метро Каховская","(55.6536252, 37.6054067)"


In [191]:
def func(s):
    s = s.replace('ё','е')
    s = s.replace('  ',' ')
    if s == 'Марк':
        s = 'Алтуфьево'
    elif s == 'Перерва':
        s = 'Братиславская'
    elif s.lower() == 'библиотека им. ленина':
        s = 'библиотека имени ленина'
    elif s.lower() == 'бескудниково':
        s = 'селигерская'
    elif s.lower() == 'гражданская':
        s = 'дмитровская'
    return s.lower().strip()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19391 entries, 0 to 19390
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        19391 non-null  int64  
 1   Адрес                     19391 non-null  object 
 2   Станция                   19391 non-null  object 
 3   Серия дома                2088 non-null   object 
 4   Общая площадь             19391 non-null  float64
 5   Площадь, м2               19391 non-null  object 
 6   Этаж                      19391 non-null  int64  
 7   Этажность дома            19391 non-null  int64  
 8   Площадь комнат, м2        12306 non-null  object 
 9   Количество комнат         18868 non-null  object 
 10  Ремонт                    16944 non-null  object 
 11  Тип                       19391 non-null  object 
 12  Дом                       19391 non-null  object 
 13  Цена за месяц             19391 non-null  int64  
 14  Залог 

In [192]:
#координаты

metro = metro.drop_duplicates('Станция').loc[:,['Станция','coor']]
metro.loc[:,'Станция'] = metro.loc[:,'Станция'].map(func)
df.loc[:,'Станция'] = df.loc[:,'Станция'].map(func)

m = df.merge(metro, how = 'left', on='Станция').dropna(how='all')
df.insert(3, "metro_coord",m['coor'])